<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN User User F1 </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, user_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_uu = knn.UserUser(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla1 = topn.RecListAnalysis()
    rla2 = topn.RecListAnalysis()
    rla1.add_metric(topn.precision)
    rla2.add_metric(topn.recall)
    prec = rla1.compute(all_recs, test_data)
    rec = rla2.compute(all_recs, test_data)
    F1 = ((prec['precision'].values*rec['recall'].values*2)/(prec['precision'].values+rec['recall'].values))
    F1 = np.nan_to_num(F1)
    return [F1.mean(),prec['precision'].mean(),rec['recall'].mean()]


In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric[0],metric[1],metric[2]],',')
    return {'loss': -metric[0], 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[36.80522439881056, 'False', 0.04486054508933237, 0.036164927968206945, 0.10538446421376259]                           
,                                                                                                                      
[3.0477312488618806, 'False', 0.033071162110679814, 0.0274904754016898, 0.07368471400492718]                           
,                                                                                                                      
[24.792438916650124, 'True', 0.04396480805366622, 0.0356946514323566, 0.10221139941791461]                             
,                                                                                                                      
[26.002830056555545, 'True', 0.04430893533463017, 0.03575000000000022, 0.10430736400630508]                            
,                                                                                                                      
[51.01075870500427, 'True', 0.0458643088

[79.17460661967925, 'True', 0.04655048765340227, 0.03715231788079484, 0.11211564222764736]                             
,                                                                                                                      
[58.77649296197939, 'True', 0.046686881612765435, 0.037355960264900796, 0.11210144120038758]                           
,                                                                                                                      
[40.841223279101975, 'True', 0.04557244550217424, 0.03668708609271543, 0.10687819051826776]                            
,                                                                                                                      
[1.8134773836922093, 'True', 0.023733649508802614, 0.020322847682119595, 0.049861493310392545]                         
,                                                                                                                      
[99.84232575505254, 'True', 0.0470551367

KeyboardInterrupt: 

In [4]:
array = [[66.04969755885027, 'True', 0.04687892461803699, 0.03744162940884284, 0.11292488453904649]   #208.13/59                          
,                                                                                                                      
[26.306328172245294, 'True', 0.044418998415476384, 0.035908262957443475, 0.1044673108698317]          #196.76/44                 
,                                                                                                                      
[25.46452974417527, 'False', 0.04434423944211034, 0.03600099354197737, 0.10319236191658489]                            
,                                                                                                                      
[20.182601266750112, 'False', 0.04358764990334716, 0.035394170255051524, 0.10039173132452274]                          
,                                                                                                                      
[43.27366501402109, 'False', 0.04579614028887878, 0.036708609271523356, 0.10920284582792304]                           
,                                                                                                                      
[9.847284412600008, 'True', 0.040258066629422065, 0.03305182977314146, 0.09187849795554606]                            
,                                                                                                                      
[14.553285202969606, 'False', 0.04208966293171185, 0.034325219407186705, 0.09638461251001496]                          
,                                                                                                                      
[58.97626340172866, 'False', 0.04639542771591558, 0.037097682119205375, 0.11179792144247795]                           
,                                                                                                                      
[22.089706413898075, 'True', 0.04407840315727065, 0.03570293094883292, 0.10237656702029228]                            
,                                                                                                                      
[88.69605126920027, 'True', 0.0470863783147936, 0.03748095395826453, 0.11471794795371623]                              
,                                                                                                                      
[7.770222600843278, 'False', 0.03819616642702076, 0.03141390728476835, 0.08676750728187145]                            
,                                                                                                                      
[58.869584133106635, 'True', 0.0460339192263695, 0.036875827814569856, 0.11034556903140416]                            
,                                                                                                                      
[50.7019134207582, 'True', 0.04681948654507371, 0.0375476072197386, 0.11185104037587038]                               
,                                                                                                                      
[29.468404578417186, 'True', 0.045313723324225945, 0.03659933774834457, 0.10537049512939603]                           
,                                                                                                                      
[32.521144400549886, 'False', 0.04553327402396215, 0.03661533366451413, 0.10789283505170515]                           
,                                                                                                                      
[15.205979390221332, 'True', 0.04252404239016177, 0.03457601854918863, 0.09815383977362531]                            
,                                                                                                                      
[86.90959476029776, 'True', 0.04721526864402055, 0.03755754263951005, 0.11496279737912603]                             
,                                                                                                                      
[59.14175966414186, 'True', 0.046678288448251046, 0.03734105960264926, 0.11138211075863452]                            
,                                                                                                                      
[25.212566678664057, 'False', 0.04444778960793472, 0.03594039735099361, 0.10391079334648258]                           
,                                                                                                                      
[3.3789218842220596, 'False', 0.03301747709632394, 0.027532704090081413, 0.0735026318627892]                           
,                                                                                                                      
[93.50186607736768, 'True', 0.04733266044141565, 0.03769956939383926, 0.11559250153697993]                             
,                                                                                                                      
[97.44928745869812, 'True', 0.04612702182254932, 0.036683225699619286, 0.1123775939568561]                             
,                                                                                                                      
[78.63306068137658, 'True', 0.0467412109128504, 0.03725165562913925, 0.11419070998187873]                              
,                                                                                                                      
[80.59314829490295, 'True', 0.04733657180527744, 0.03772644477562542, 0.11462663389100046]                             
,                                                                                                                      
[98.64568223712511, 'True', 0.04670415272633756, 0.03709837694600892, 0.11407072473448121]                             
,                                                                                                                      
[76.41960673735528, 'True', 0.04754277395066709, 0.037913907284768365, 0.1156983709613924]                             
,                                                                                                                      
[75.76413649465584, 'True', 0.04685230899685276, 0.03736920529801342, 0.11394618204807137]                             
,                                                                                                                      
[74.22162466813337, 'True', 0.0471922550675867, 0.0377161311692616, 0.11409197785072767]                               
,                                                                                                                      
[67.87726979909016, 'True', 0.04670316861721832, 0.03726610365954655, 0.11283103158513358]                             
,                                                                                                                      
[84.198137264314, 'True', 0.04698249632422208, 0.03744370860927178, 0.11363506476511727]                               
,                                                                                                                      
[69.69209544337684, 'True', 0.04682506470271845, 0.037302533532042076, 0.11416194698251592]                            
,                                                                                                                      
[80.77616025956036, 'True', 0.046849737460832486, 0.037312913907284875, 0.11389656367081277]                           
,                                                                                                                      
[40.40840598945657, 'True', 0.04575792175977147, 0.03670255051341529, 0.10888263701561121]                             
,                                                                                                                      
[53.58095239553812, 'True', 0.04697887076019388, 0.03754304635761609, 0.11214139997193222]                             
,                                                                                                                      
[93.24252863736596, 'True', 0.04680406677560141, 0.03722967378705093, 0.11381624270945483]                             
,                                                                                                                      
[64.05841930554647, 'True', 0.04595219259670255, 0.03674283821824831, 0.11041555706313992]                             
,                                                                                                                      
[72.70225311930956, 'False', 0.04689287306685747, 0.037473091571452406, 0.11338841566649527]                           
,                                                                                                                      
[42.54890102390017, 'True', 0.04627203675077079, 0.037138125207022546, 0.11065031220608587]                            
,                                                                                                                      
[62.87504826824842, 'True', 0.04688303649462177, 0.037526490066225375, 0.11257910273153629]                            
,                                                                                                                      
[81.17504590671439, 'False', 0.046646415619485675, 0.03731291390728505, 0.11249822956685947]                           
,                                                                                                                      
[91.14271103016952, 'True', 0.04725962619552925, 0.03757410167246257, 0.1155035659212219]                              
,                                                                                                                      
[46.604260473532655, 'True', 0.04582894162182305, 0.036779801324503515, 0.10859325931231609]                           
,                                                                                                                      
[36.995148626486724, 'False', 0.04578709809745335, 0.03674668874172215, 0.10879953072980286]                           
,                                                                                                                      
[99.9384693754923, 'True', 0.046419029248966015, 0.037016062261964154, 0.112118268493345]                              
,                                                                                                                      
[86.28754489185125, 'True', 0.04711006252096031, 0.037413050679033055, 0.11418673576925614]                            
,                                                                                                                      
[54.5351261954339, 'False', 0.046500755109575145, 0.037140255009107705, 0.11194414011275924]                           
,                                                                                                                      
[70.60421136794795, 'True', 0.04718800045305253, 0.037655240933929766, 0.11402032572930754]                            
,                                                                                                                      
[60.99360051110712, 'True', 0.04674112037470646, 0.03737704918032817, 0.11179369084190965]                             
,                                                                                                                      
[47.07249547529426, 'False', 0.045957007636156, 0.03690511674118265, 0.10910593284352897]                              
,                                                                                                                      
[66.14174419031775, 'True', 0.04611858291434437, 0.0368923841059605, 0.1110202643209041]                               
,                                                                                                                      
[83.61172992948903, 'True', 0.046620466072822775, 0.03721026490066258, 0.1130360357485205]                             
,                                                                                                                      
[77.84889945131218, 'False', 0.04686612309404654, 0.03734514739980139, 0.11371250728976096]                            
,                                                                                                                      
[55.26526277506797, 'True', 0.046497048487698416, 0.03717243664071577, 0.1116639415763178]                             
,                                                                                                                      
[96.52456654025119, 'True', 0.047086153474957475, 0.037537671799967275, 0.11388552494206218]                           
,                                                                                                                      
[91.0472083561265, 'True', 0.046589014643736396, 0.03712439953619373, 0.11308401642141043]                             
,                                                                                                                      
[88.51188936274428, 'False', 0.046655080467025864, 0.037122516556291604, 0.11379421977428214]                          
,                                                                                                                      
[76.3169842728994, 'True', 0.04684331812853625, 0.037325716178175455, 0.11389097044620342]                             
,                                                                                                                      
[71.5412276718487, 'True', 0.04608110157424967, 0.03678807947019883, 0.11118853845810056]                              
,                                                                                                                      
[48.686510758632906, 'True', 0.04649448881735241, 0.037205298013245125, 0.11155891144291963] 
,
        [36.80522439881056, 'False', 0.04486054508933237, 0.036164927968206945, 0.10538446421376259]                           
,                                                                                                                      
[3.0477312488618806, 'False', 0.033071162110679814, 0.0274904754016898, 0.07368471400492718]                           
,                                                                                                                      
[24.792438916650124, 'True', 0.04396480805366622, 0.0356946514323566, 0.10221139941791461]                             
,                                                                                                                      
[26.002830056555545, 'True', 0.04430893533463017, 0.03575000000000022, 0.10430736400630508]                            
,                                                                                                                      
[51.01075870500427, 'True', 0.0458643088879592, 0.03681020205366045, 0.10928703585938289]                              
,                                                                                                                      
[47.910489996660814, 'False', 0.045790083458238554, 0.0367191122888376, 0.10855624534911418]                           
,                                                                                                                      
[47.80265640808453, 'False', 0.04633667802699793, 0.037117072362974256, 0.11096003821348041]                           
,                                                                                                                      
[52.53425857909284, 'False', 0.04617719849767392, 0.03705629139072869, 0.11007416985111147]                            
,                                                                                                                      
[83.73515002757296, 'False', 0.046834916351056924, 0.03741305067903308, 0.11268617444918881]                           
,                                                                                                                      
[33.16720778716683, 'True', 0.04594875209351076, 0.03693543046357635, 0.10934787858715068]                             
,                                                                                                                      
[71.61786232344575, 'False', 0.04628147647304799, 0.036997847325716336, 0.11137424305078424]                           
,                                                                                                                      
[83.02813921564952, 'False', 0.04629378950136266, 0.03701771816525939, 0.11238966578013376]                            
,                                                                                                                      
[54.86297699539446, 'True', 0.04580511616927559, 0.03665120900960607, 0.10981214053371983]                             
,                                                                                                                      
[39.836044530677256, 'False', 0.04562248687704148, 0.036744494121543486, 0.10781696428309606]                          
,                                                                                                                      
[97.01946261662053, 'False', 0.04705856896765335, 0.037486754966887643, 0.11490194840993839]                           
,                                                                                                                      
[65.52331033592966, 'True', 0.04637912404208192, 0.03707733068388829, 0.11111371203070246]                             
,                                                                                                                      
[91.43324700466412, 'True', 0.047055726626990924, 0.03745860927152332, 0.11456828517797456]                            
,                                                                                                                      
[17.965155249035018, 'False', 0.04314831844648293, 0.03494783904619988, 0.1011276623771577]                            
,                                                                                                                      
[72.74473319846024, 'False', 0.046908427798801375, 0.037396920019871153, 0.11398610566190806]                          
,                                                                                                                      
[17.064866964497526, 'False', 0.04333004057997467, 0.035219407186620505, 0.10030496198574948]                          
,                                                                                                                      
[99.41249502548541, 'True', 0.04671940939079777, 0.037103825136612235, 0.11318917145972072]                            
,                                                                                                                      
[99.74468659354145, 'True', 0.04739762855868138, 0.03764657171248788, 0.11571871193254749]                             
,                                                                                                                      
[99.26029500769395, 'True', 0.04687475027296443, 0.037293046357616245, 0.11411494869159299]                            
,                                                                                                                      
[90.75440815842893, 'True', 0.04698605946081547, 0.03743211920529833, 0.1143797838575035]                              
,                                                                                                                      
[99.45701465838425, 'True', 0.047547319975795276, 0.037825438887048896, 0.11592096243320119]                           
,                                                                                                                      
[81.46067673059702, 'True', 0.046914933999823244, 0.03741967538920187, 0.11305595055812644]                            
,                                                                                                                      
[62.85298471062691, 'True', 0.04679961562451341, 0.037506209637357464, 0.11217142008332777]                            
,                                                                                                                      
[90.46644358959684, 'True', 0.046815769548312874, 0.037271071369432356, 0.11442253531032633]                           
,                                                                                                                      
[73.40097749532794, 'True', 0.04724252151967767, 0.037746315615168405, 0.11307697006330181]                            
,                                                                                                                      
[77.13957926105877, 'True', 0.0465197676826003, 0.03711376055638374, 0.11287542723521145]                              
,                                                                                                                      
[64.62567045754454, 'True', 0.04661591454623242, 0.03730915714522298, 0.11170435652722645]                             
,                                                                                                                      
[95.23783676241416, 'True', 0.047202035592491164, 0.03756456953642409, 0.11483901182516291]                            
,                                                                                                                      
[3.788009691976157, 'True', 0.03274216620020664, 0.027274834437086427, 0.07222781598489002]                            
,                                                                                                                      
[86.55685997067869, 'True', 0.04652065318512918, 0.03711943018055355, 0.11281297890735788]                             
,                                                                                                                      
[79.17460661967925, 'True', 0.04655048765340227, 0.03715231788079484, 0.11211564222764736]                             
,                                                                                                                      
[58.77649296197939, 'True', 0.046686881612765435, 0.037355960264900796, 0.11210144120038758]                           
,                                                                                                                      
[40.841223279101975, 'True', 0.04557244550217424, 0.03668708609271543, 0.10687819051826776]                            
,                                                                                                                      
[1.8134773836922093, 'True', 0.023733649508802614, 0.020322847682119595, 0.049861493310392545]                         
,                                                                                                                      
[99.84232575505254, 'True', 0.04705513671303467, 0.03742795627691316, 0.11454973777675284]                             
,                                                                                                                      
[87.96143913439106, 'True', 0.04676620268656794, 0.037273178807947276, 0.11398904671410343]                            
,                                                                                                                      
[93.56922170720938, 'True', 0.04676459636127805, 0.037293805895992356, 0.11358741673905422] ]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','F1','Precision','Recall']
metric.head(5)

,neighbours,center,F1,Precision,Recall
0,66.049698,True,0.046879,0.037442,0.112925
1,26.306328,True,0.044419,0.035908,0.104467
2,25.464530,False,0.044344,0.036001,0.103192
3,20.182601,False,0.043588,0.035394,0.100392
4,43.273665,False,0.045796,0.036709,0.109203


In [6]:
metric.sort_values(by=['F1'],ascending=False).head(10)

,neighbours,center,F1,Precision,Recall
83,99.457015,True,0.047547,0.037825,0.115921
25,76.419607,True,0.047543,0.037914,0.115698
80,99.744687,True,0.047398,0.037647,0.115719
23,80.593148,True,0.047337,0.037726,0.114627
20,93.501866,True,0.047333,0.037700,0.115593
40,91.142711,True,0.047260,0.037574,0.115504
87,73.400977,True,0.047243,0.037746,0.113077
16,86.909595,True,0.047215,0.037558,0.114963
90,95.237837,True,0.047202,0.037565,0.114839
27,74.221625,True,0.047192,0.037716,0.114092


In [7]:
metric.to_csv('metric_F1.csv', index=False)